#### Libraries


In [1]:
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import torch
import timm
import csv
from sklearn import svm
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
import os
import itertools

# 3D to 2D


#### MIP


In [2]:
def mip(image: np.ndarray)->np.ndarray:
    """Create the maximum intensity projection.
    """
    return np.max(image, axis=0)

#### AIP


In [3]:
def aip(image: np.ndarray)->np.ndarray:
    """Create a mean image projection.
    """
    return np.mean(image,axis=0)

#### SLICE


In [4]:
def find_tumor_center(mask: np.ndarray)->int:
    """Find the center of the tumor. 
    """
    return np.argmax(np.sum(mask, axis=(1, 2)))

In [5]:
def slice_tc(image: np.ndarray, mask: np.ndarray)->np.ndarray:
    """Take a slice at the center of the tumor.
    """
    x1 = find_tumor_center(mask)
    return image[x1, :, :]

#### TUMOR MIP

In [6]:
def tumor_mip(image: np.ndarray, mask: np.ndarray)->np.ndarray:
    """Return the mip of the tumor only.
    """
    return np.max(image * mask, axis=0)

#### MINIP

In [77]:
def minip(image: np.ndarray)->np.ndarray:
    """Create the maximum intensity projection.
    """
    return np.min(image, axis=0)

# Convert the images


In [85]:
MIP = []
AIP = []
SLICE = []
TUMOR_MIP = []
MINIP = []

with open('c:/users/sven/Documents/BEP_sbierenbroodspot_1334859/Pyradiomics/v5/folderNames2.csv', newline='') as file:
    reader = csv.reader(file)
    next(reader)
    
    for row in reader:
        img_data = sitk.GetArrayFromImage(sitk.ReadImage(row[1]))

        if os.path.exists(row[2]):
            mask_data = sitk.GetArrayFromImage(sitk.ReadImage(row[2]))
        else:
            mask_l = sitk.GetArrayFromImage(sitk.ReadImage(row[3]))
            mask_r = sitk.GetArrayFromImage(sitk.ReadImage(row[4]))
            mask_data = mask_l+mask_r

        MIP.append(mip(img_data))
        AIP.append(aip(img_data))
        SLICE.append(slice_tc(img_data,mask_data))
        TUMOR_MIP.append(tumor_mip(img_data,mask_data))
        MINIP.append(minip(img_data))

# feature extraction


In [87]:
def IAC(layout: list, i: int) -> np.ndarray:
    """
    Create the 512x512x3 input array for feature extraction.
    """
    slots = [MIP[i], AIP[i], SLICE[i], TUMOR_MIP[i], MINIP[i]]
    input_array = np.dstack([slots[layout[j] - 1] for j in range(3)])
    return input_array

#### Model


In [9]:
timm.list_models(pretrained=True)

['adv_inception_v3',
 'bat_resnext26ts',
 'beit_base_patch16_224',
 'beit_base_patch16_224_in22k',
 'beit_base_patch16_384',
 'beit_large_patch16_224',
 'beit_large_patch16_224_in22k',
 'beit_large_patch16_384',
 'beit_large_patch16_512',
 'beitv2_base_patch16_224',
 'beitv2_base_patch16_224_in22k',
 'beitv2_large_patch16_224',
 'beitv2_large_patch16_224_in22k',
 'botnet26t_256',
 'cait_m36_384',
 'cait_m48_448',
 'cait_s24_224',
 'cait_s24_384',
 'cait_s36_384',
 'cait_xs24_384',
 'cait_xxs24_224',
 'cait_xxs24_384',
 'cait_xxs36_224',
 'cait_xxs36_384',
 'coat_lite_mini',
 'coat_lite_small',
 'coat_lite_tiny',
 'coat_mini',
 'coat_tiny',
 'coatnet_0_rw_224',
 'coatnet_1_rw_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_rw_224',
 'coatnet_rmlp_1_rw_224',
 'coatnet_rmlp_2_rw_224',
 'coatnet_rmlp_nano_rw_224',
 'coatnext_nano_rw_224',
 'convit_base',
 'convit_small',
 'convit_tiny',
 'convmixer_768_32',
 'convmixer_1024_20_ks9_p14',
 'convmixer_1536_20',
 'convnext_atto',
 'convnext_atto_

In [124]:
model = timm.create_model('resnetv2_101', pretrained=True)
model.eval()
model.to('cuda')


ResNetV2(
  (stem): Sequential(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (stages): Sequential(
    (0): ResNetStage(
      (blocks): Sequential(
        (0): PreActBottleneck(
          (downsample): DownsampleConv(
            (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (norm): Identity()
          )
          (norm1): BatchNormAct2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (norm2): BatchNormAct2d(
            64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): ReLU(inplace=True)
          )
          (conv2): Conv2d(64, 64, kernel_size=

#### Feature extraction single image

In [64]:
def feature_extraction(input_array: np.ndarray)-> np.ndarray:
    """Extract the features of a single image.
    """ 
    
    # Convert numpy array to torch tensor and normalize
    input_tensor = torch.from_numpy(np.transpose(input_array, (2, 0, 1))).float()
    input_tensor = input_tensor.to('cuda')
    input_tensor /= 255.0
    input_tensor = input_tensor.unsqueeze(0)

    # Extract features using ResNet50
    with torch.no_grad():
        features = model.forward_features(input_tensor)
        features = torch.mean(features, dim=[2, 3])

    # Convert features to numpy array
    features = features.to('cpu')
    features_array = features.numpy()
    
    return features_array

#### Feature extraction full dataset

In [125]:
def feature_extraction_full(layout: list) -> np.ndarray:
    """Extract all features of the dataset for an input layout.
    """
    result = np.empty((0, 2048))

    for i in range(len(MIP)):
        result_vector = feature_extraction(IAC(layout, i))
        result = np.vstack([result, result_vector])    
    return result

#### Extract manual result


In [126]:
result  = feature_extraction_full([2, 1, 5])
print(svm_auc_max(result, labels))
np.savetxt("result.csv", result, delimiter=",")

0.7304040404040404


# Labels


In [14]:
labels = np.empty((0,0))
with open('c:/users/sven/Documents/BEP_sbierenbroodspot_1334859/Technical_research/Labels.csv', newline='') as file:
    reader = csv.reader(file)
    next(reader)
    for row in reader:
        labels = np.append(labels, row[0])

# SVM


In [15]:
def svm_auc(data: np.ndarray, labels: np.ndarray)->np.float64:
    """Get the AUC for a svm.
    """

    # Create a stratified 5-fold cross-validation object
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    # Initialize an SVM classifier
    svm = SVC(kernel='linear', probability=True)

    # Store the AUC scores
    auc_scores = []

    # Perform cross-validation
    for train_idx, test_idx in cv.split(data, labels):
        # Split the data and labels
        X_train, X_test = data[train_idx], data[test_idx]
        y_train, y_test = labels[train_idx], labels[test_idx]

        # Train the SVM classifier
        svm.fit(X_train, y_train)

        # Predict the probabilities for the test set
        y_pred_proba = svm.predict_proba(X_test)[:, 1]

        # Compute the AUC score
        auc_score = roc_auc_score(y_test, y_pred_proba)

        # Store the AUC score
        auc_scores.append(auc_score)

    return np.mean(auc_scores)

In [16]:
def svm_auc_max(data: np.ndarray, labels: np.ndarray)->np.float64:
    """Get the max AUC out of a constant amount of tries.
    """
    # Initialize a variable to store the maximum output
    max_output = -np.inf

    # Iterate over the function 20 times
    for i in range(100):
        # Compute the output of the function
        output = svm_auc(data, labels)

        # Update the maximum output if necessary
        if output > max_output:
            max_output = output
    return max_output

# Get all results

In [121]:
auc = []

for i in range(1, 5):
    for j in range(1, 5):
        for k in range(1, 5):
            my_list = [i, j, k]
            auc_ = svm_auc_max(feature_extraction_full(my_list),labels)
            auc.append(auc_)
            if auc_ > 0.7:
                print(my_list, auc_)



In [76]:
np.savetxt("aucs.csv", auc, delimiter=",")

# Results for 5 configs

In [122]:

for i in range(1, 6):
    for j in range(1, 6):
        for k in range(1, 6):
            if i == 5 or j == 5 or k == 5:
                my_list = [i, j, k]
                auc_ = svm_auc_max(feature_extraction_full(my_list),labels)
                auc.append(auc_)
                if auc_ > 0.7:
                    print(my_list, auc_)

[4, 5, 5] 0.7056565656565656
[5, 5, 5] 0.7002020202020202


In [123]:
np.savetxt("aucs.csv", auc, delimiter=",")

# String generation

In [91]:
with open('my_file.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    for i in range(1, 6):
        for j in range(1, 6):
            for k in range(1, 6):
                if i == 5 or j == 5 or k == 5:
                    my_list = [i, j, k]
                    if 5 in my_list:
                        csvwriter.writerow([str(my_list)])